In [2]:
import sys, os, glob, time
sys.path.append("../src")
os.environ["MPLBACKEND"] = "Agg"  # avoids matplotlib backend issues from version changes

start_time = time.time()

import cv2
from otsu import otsu_threshold
from contrast_color import clahe_preserve_color
from contour_crop_2 import contour_crop_binary
from homomorphic_filter import homomorphic_filter_color
from wavelet import wavelet_denoise_lab_cv

RAW_DIR = os.path.abspath("../data/raw_images")
OUT_DIR = os.path.abspath("../data/processed_images")
SUFFIX  = "_processed_piplinetest12"

os.makedirs(OUT_DIR, exist_ok=True)

# gather images
exts = ("*.jpg","*.JPG","*.jpeg","*.JPEG","*.png","*.PNG","*.tif","*.tiff","*.bmp")
files = [p for e in exts for p in glob.glob(os.path.join(RAW_DIR, e))]
files.sort()

if not files:
    print(f"No images found in {RAW_DIR}")
    raise SystemExit(0)

ok = fail = 0
for in_path in files:
    try:
        # 1) Homomorphic filtering
        hf_img, fname = homomorphic_filter_color(in_path)
        
        # 2) CLAHE
        clahe_img, _ = clahe_preserve_color(hf_img, fname)

        # 3) Wavelet Denoising
        wave_img, _ = wavelet_denoise_lab_cv(clahe_img, fname=fname)

        # 3) Otsu
        binary_t, _ = otsu_threshold(clahe_img, fname)

        # 4) Contour crop
        cropped_img, _ = contour_crop_binary(binary_t, wave_img, fname)

        # 4) Save with same name + suffix
        base, ext = os.path.splitext(fname)
        out_name = f"{base}{SUFFIX}{ext or '.jpg'}"
        out_path = os.path.join(OUT_DIR, out_name)
        if not cv2.imwrite(out_path, cropped_img):
            raise RuntimeError("cv2.imwrite returned False")
        ok += 1
        print(f"[OK] {os.path.basename(in_path)} -> {out_name}")
    except Exception as e:
        fail += 1
        print(f"[FAIL] {os.path.basename(in_path)}: {e}")

print(f"\nDone. Saved {ok}. Failed {fail}. Output: {OUT_DIR}")

elapsed = time.time() - start_time
print(f"\nTotal processing time: {elapsed: 2f} seconds")

[OK] T0004-04-06-2019_BL (2).JPG -> T0004-04-06-2019_BL (2)_processed_piplinetest12.JPG
[OK] T0011-06-06-2019_N.JPG -> T0011-06-06-2019_N_processed_piplinetest12.JPG
[OK] T0013-06-06-2019_BL (1).JPG -> T0013-06-06-2019_BL (1)_processed_piplinetest12.JPG
[OK] T0013-06-06-2019_N.JPG -> T0013-06-06-2019_N_processed_piplinetest12.JPG
[OK] T0017_10-06-2019 (1).JPG -> T0017_10-06-2019 (1)_processed_piplinetest12.JPG
[OK] T0019_2019-06-10_BL (1).JPG -> T0019_2019-06-10_BL (1)_processed_piplinetest12.JPG
[OK] T0026_13-06-2019_N.JPG -> T0026_13-06-2019_N_processed_piplinetest12.JPG
[OK] T0027_13-06-2019_N.JPG -> T0027_13-06-2019_N_processed_piplinetest12.JPG
[OK] T0028_13-06-2019_BL (2).JPG -> T0028_13-06-2019_BL (2)_processed_piplinetest12.JPG
[OK] T0036_17-06-2019_N.JPG -> T0036_17-06-2019_N_processed_piplinetest12.JPG
[OK] T0040_18-06-2019_N.JPG -> T0040_18-06-2019_N_processed_piplinetest12.JPG
[OK] T0043_19-06-2019_N.JPG -> T0043_19-06-2019_N_processed_piplinetest12.JPG
[OK] T0044_19-06-201